In [1]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, mean_squared_error

In [2]:
# Load the dictionary from the pickle file
with open('dictionary.pkl', 'rb') as file:
    loaded_dictionary = pickle.load(file)

# View the contents of the loaded dictionary
for key, values in loaded_dictionary.items():
    product_id = key
    aisle_id, department_id = values
    print(f"Product ID: {product_id}, Aisle ID: {aisle_id}, Department ID: {department_id}")

Product ID: 1, Aisle ID: 1, Department ID: 16
Product ID: 2, Aisle ID: 2, Department ID: 4
Product ID: 3, Aisle ID: 3, Department ID: 13
Product ID: 4, Aisle ID: 4, Department ID: 16
Product ID: 5, Aisle ID: 4, Department ID: 16
Product ID: 6, Aisle ID: 5, Department ID: 4
Product ID: 7, Aisle ID: 2, Department ID: 4
Product ID: 8, Aisle ID: 5, Department ID: 4
Product ID: 9, Aisle ID: 6, Department ID: 17
Product ID: 10, Aisle ID: 2, Department ID: 4
Product ID: 11, Aisle ID: 7, Department ID: 1
Product ID: 12, Aisle ID: 8, Department ID: 7
Product ID: 13, Aisle ID: 9, Department ID: 14
Product ID: 14, Aisle ID: 5, Department ID: 4
Product ID: 15, Aisle ID: 2, Department ID: 4
Product ID: 16, Aisle ID: 10, Department ID: 4
Product ID: 17, Aisle ID: 2, Department ID: 4
Product ID: 18, Aisle ID: 11, Department ID: 4
Product ID: 19, Aisle ID: 12, Department ID: 19
Product ID: 20, Aisle ID: 13, Department ID: 15
Product ID: 21, Aisle ID: 14, Department ID: 1
Product ID: 22, Aisle ID: 15, 

In [23]:
# Load the prod_dict from the pickle file
with open('prod_dict.pkl', 'rb') as file1:
    prod_dictionary = pickle.load(file1)

# View the contents of the loaded dictionary
for key, values in prod_dictionary.items():
    product_id = key
    product_name = values
    print(f"Product ID: {product_id}, Product Name: {product_name}")

Product ID: 1, Product Name: ['Organic Egg Whites']
Product ID: 2, Product Name: ['Carrots']
Product ID: 3, Product Name: ['Garlic Powder']
Product ID: 4, Product Name: ['Unsweetened Chocolate Almond Breeze Almond Milk']
Product ID: 5, Product Name: ['Unsweetened Almondmilk']
Product ID: 6, Product Name: ['Lemons']
Product ID: 7, Product Name: ['Organic Ginger Root']
Product ID: 8, Product Name: ['Organic Baby Spinach']
Product ID: 9, Product Name: ['One Ply Choose A Size Big Roll Paper Towel Rolls']
Product ID: 10, Product Name: ['Artichokes']
Product ID: 11, Product Name: ['Pineapple Chunks']
Product ID: 12, Product Name: ['Orange Juice']
Product ID: 13, Product Name: ['Fruit & Nutty Almonds Raisins Cranberries Pecans Granola']
Product ID: 14, Product Name: ['Baby Spinach']
Product ID: 15, Product Name: ['Green Beans']
Product ID: 16, Product Name: ['Organic Cilantro']
Product ID: 17, Product Name: ['Organic Butterhead (Boston, Butter, Bibb) Lettuce']
Product ID: 18, Product Name: ['

In [3]:
# Load the saved model
loaded_model = load_model("savedmodel/")

In [25]:
# Taking user inputs

# dow = input("Enter the week from 0 to 7")
# hour_of_day = input("Enter the hour of day from 0 to 23")

dow = 6
hour_of_day = 14

In [26]:
dict = {
    'product_id': [],
    'order_dow': [],
    'order_hour_of_day': [],
    'aisle_id':[],
    'department_id': []
}

In [27]:
for key, (value1, value2) in loaded_dictionary.items():
    dict['product_id'].append(key)
    dict['order_dow'].append(dow)
    dict['order_hour_of_day'].append(hour_of_day)
    dict['aisle_id'].append(value1)
    dict['department_id'].append(value2)

In [28]:
for key in dict:
    dict[key] = np.array(dict[key]).astype('int64')

In [29]:
# Use the loaded model for predictions
pred_ans = loaded_model.predict(dict, batch_size=256)

12/12 [==============================] - 0s 4ms/step


In [30]:
print(pred_ans)

[[0.9999322 ]
 [0.7381213 ]
 [0.58699805]
 ...
 [0.09958195]
 [0.48062596]
 [0.4801382 ]]


In [31]:
# Selecting indices of the items with highest recommendation values
# Flatten the array to get a 1-dimensional array
flattened_arr = pred_ans.flatten()

# Get the indices that would sort the array in descending order
sorted_indices = np.argsort(flattened_arr)[::-1]

# Get the top 10 indices
top_10_indices = sorted_indices[:10]

# Print the top 10 indices
print(top_10_indices)

[  80  413 1266  381  101  399  401  404   89  414]


In [32]:
# Recommending items
for index in top_10_indices:
    if index in prod_dictionary:
        prod_name = prod_dictionary[index]
        print(f"Product {index}: {prod_name}")
    else:
        print(f"Product {index}: Not found in dictionary")

Product 80: ['Smartwater']
Product 413: ['Organic Low Fat Vegetable Barley Soup']
Product 1266: ['Roasted Chicken Breast']
Product 381: ['Cold-Pressed Organic Grapefruit']
Product 101: ['Wint-O-Green']
Product 399: ['Romaine Hearts']
Product 401: ['Organic Lacinato (Dinosaur) Kale']
Product 404: ['Original Real Vegetable Chips']
Product 89: ['Heirloom Tomato']
Product 414: ['Island Vanilla Cereal']
